In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "hw3.png")

In [2]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    #print(header)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [4]:
Image(url= "problem1.png")

In [5]:
data = []
user_per_recipe = defaultdict(set)
recipe_per_user = defaultdict(set)

for user,recipe,d in readCSV("trainInteractions.csv.gz"):
    data.append([user,recipe])
    recipe_per_user[user].add(recipe)
    user_per_recipe[recipe].add(user)

train = data[:400000]
valid = data[400000:]

In [6]:
print(len(valid))
#print(ratings)

100000


In [7]:
import random #this block takes so long to run
recipe_set = set(recipe_per_user.keys())
cnt = 0
valid_add = []
for user,recipe in valid :
    #print(user, recipe)
    if cnt%20000==0:
        print(cnt)
    cnt+=1
    false_set = recipe_set - recipe_per_user[user]
    recipe_false = random.sample(list(false_set), 1)
    #print(recipe_false)
    valid_add.append([user, recipe_false[0]])
valid += valid_add
print(len(valid))

0
20000
40000
60000
80000
200000


In [13]:
### Would-cook baseline: just rank which recipes are popular and which are not, and return '1' if a recipe is among the top-ranked

recipeCount = defaultdict(int)
totalCooked = 0

for user,recipe,_ in readCSV("trainInteractions.csv.gz"):
  recipeCount[recipe] += 1
  totalCooked += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()
# high to low
#print(mostPopular)

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalCooked/2: break

y_pred = []
cnt=0
for user,recipe in valid :
    #print(recipe)
    if cnt%20000==0:
        print(cnt)
    cnt+=1
    if recipe in return1:
        y_pred.append(1)
    else :
        y_pred.append(0)
        
y_valid = [1]*100000+[0]*100000

TP = sum([(p and l) for (p,l) in zip(y_pred, y_valid)])
FP = sum([(p and not l) for (p,l) in zip(y_pred, y_valid)])
TN = sum([(not p and not l) for (p,l) in zip(y_pred, y_valid)])
FN = sum([(not p and l) for (p,l) in zip(y_pred, y_valid)])
ACC = (TP + TN) / (TP + FP + TN + FN)
print(ACC)

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
0.69627


Ans for Problem1 :

a. accuracy of the baseline model on validation set = 0.696805

In [14]:
Image(url= "problem2.png")

In [15]:
def predict_by_threshold(percentile):
    return1 = set()
    count = 0
    for ic, i in mostPopular:
      count += ic
      return1.add(i)
      if count > totalCooked*percentile/100: break
    y_pred = []
    for u,i in valid :
        if i in return1:
            y_pred.append(1)
        else :
            y_pred.append(0)
    TP = sum([(p and l) for (p,l) in zip(y_pred, y_valid)])
    FP = sum([(p and not l) for (p,l) in zip(y_pred, y_valid)])
    TN = sum([(not p and not l) for (p,l) in zip(y_pred, y_valid)])
    FN = sum([(not p and l) for (p,l) in zip(y_pred, y_valid)])
    ACC = (TP + TN) / (TP + FP + TN + FN)
    print("percentile : ",percentile,", ACC = ",ACC)
    
for percentile in range(10,100,10):
    predict_by_threshold(percentile)

percentile :  10 , ACC =  0.548565
percentile :  20 , ACC =  0.59391
percentile :  30 , ACC =  0.634985
percentile :  40 , ACC =  0.66807
percentile :  50 , ACC =  0.69627
percentile :  60 , ACC =  0.723925
percentile :  70 , ACC =  0.754945
percentile :  80 , ACC =  0.783615
percentile :  90 , ACC =  0.8065


Ans for Problem2 :

a. accuracy of the best threshold model on validation set = 0.80843
   with threshold = 90

In [16]:
Image(url= "problem3.png")

In [17]:
# Jaccard similarity interchanging users and items
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

# prediction based on Jaccard Similarity
def predict_by_jaccard(threshold):
    y_pred = []
    y_valid = [1]*100000+[0]*100000
    for u,i in valid :
        items = recipe_per_user[u]
        s1 = user_per_recipe[i]
        jaccard_sim = [0]
        for g in items-{i} :
            s2 = user_per_recipe[g]
            jaccard_sim.append(Jaccard(s1, s2))
        #print(jaccard_sim)
        if max(jaccard_sim)> threshold/100:
            y_pred.append(1)
        else:
            y_pred.append(0)
            
    TP = sum([(p and l) for (p,l) in zip(y_pred, y_valid)])
    FP = sum([(p and not l) for (p,l) in zip(y_pred, y_valid)])
    TN = sum([(not p and not l) for (p,l) in zip(y_pred, y_valid)])
    FN = sum([(not p and l) for (p,l) in zip(y_pred, y_valid)])
    ACC = (TP + TN) / (TP + FP + TN + FN)
    print("threshold : ",threshold,", ACC = ",ACC)

for threshold in range(10,100,10):
    predict_by_jaccard(threshold)


threshold :  10 , ACC =  0.8077
threshold :  20 , ACC =  0.75093
threshold :  30 , ACC =  0.70707
threshold :  40 , ACC =  0.655665
threshold :  50 , ACC =  0.595455
threshold :  60 , ACC =  0.59426
threshold :  70 , ACC =  0.58028
threshold :  80 , ACC =  0.57944
threshold :  90 , ACC =  0.57944


Ans for Problem3 :

a. accuracy of the best jaccard model on validation set = 0.71121
   with threshold = 0.1

In [18]:
Image(url= "problem4.png")

In [19]:
def predict_by_combine(percentile, threshold):
    return1 = set()
    count = 0
    for ic, i in mostPopular:
      count += ic
      return1.add(i)
      if count > totalCooked*percentile/100: break
    y_pred = []
    for u,i in valid :
        if i in return1:
            y_pred.append(1)
        else :
            y_pred.append(0)
            
    TP = sum([(p and l) for (p,l) in zip(y_pred, y_valid)])
    FP = sum([(p and not l) for (p,l) in zip(y_pred, y_valid)])
    TN = sum([(not p and not l) for (p,l) in zip(y_pred, y_valid)])
    FN = sum([(not p and l) for (p,l) in zip(y_pred, y_valid)])
    ACC = (TP + TN) / (TP + FP + TN + FN)
    print("percentile : ",percentile,", ACC = ",ACC)

Ans for Problem4 :

a. 

In [20]:
Image(url= "problem5.png")

In [21]:
# train the model on train + validation and predict
def write_prediction_most_popular(threshold):
    predictions = open("predictions_Made_popular_%d.txt" % threshold,'w')
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalCooked*threshold/100: break

    for l in open("stub_Made.txt"):
      if l.startswith("user_id"):
        #header
        predictions.write(l)
        continue
      u,i = l.strip().split('-')
      if i in return1:
        predictions.write(u + '-' + i + ",1\n")
      else:
        predictions.write(u + '-' + i + ",0\n")

    predictions.close()

for threshold in [50,60,70,80,90]:
    write_prediction_most_popular(threshold)

In [22]:
# train the model on train + validation and predict
def write_prediction_jaccard(threshold):
    predictions = open("predictions_Made_jaccard_%d.txt"%threshold, 'w')

    for l in open("stub_Made.txt"):
      if l.startswith("user_id"):
        #header
        predictions.write(l)
        continue
      u,i = l.strip().split('-')
      items = recipe_per_user[u]
      s1 = user_per_recipe[i]
      jaccard_sim = [0]
      for g in items-{i} :
            s2 = user_per_recipe[g]
            jaccard_sim.append(Jaccard(s1, s2))
      #print(jaccard_sim)
      if max(jaccard_sim)> threshold/100:
            predictions.write(u + '-' + i + ",1\n")
      else:
            predictions.write(u + '-' + i + ",0\n")  

    predictions.close()

for threshold in [10,20]:
    write_prediction_jaccard(threshold)

Ans for Problem5 :

a. accuracy of the most popular 50 model on validation set = 0.68660
   accuracy of the most popular 60 model on validation set = 0.69930
   accuracy of the most popular 70 model on validation set = 0.68480
   accuracy of the most popular 80 model on validation set = 0.70230
   accuracy of the most popular 90 model on validation set = 0.63180
   accuracy of the jaccard threshold 10 model on validation set = 0.52500
   accuracy of the jaccard threshold 20 model on validation set = 0.50849

In [23]:
Image(url= "problem9.png")

In [8]:
rating_data = []
user_per_recipe = defaultdict(set)
recipe_per_user = defaultdict(set)

for user,recipe,d in readCSV("trainInteractions.csv.gz"):
    rating_data.append([user,recipe,int(d['rating'])])
    recipe_per_user[user].add(recipe)
    user_per_recipe[recipe].add(user)

rating_train = rating_data[:400000]
rating_valid = rating_data[400000:]

N = len(rating_data)
nUsers = len(list(recipe_per_user.keys()))
nItems = len(list(user_per_recipe.keys()))
users = list(recipe_per_user.keys())
items = list(user_per_recipe.keys())
#print(nUsers, nItems)
#print(len(users),len(items))
#print(len(users)+len(items))

In [9]:
import numpy as np
alpha = np.mean(np.array([d[2] for d in rating_data]))
print(alpha)

4.580794


In [10]:
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [11]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

In [12]:
def unpack(theta):
    #print("unpack, len(theta) = ", len(theta))
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    #print(users)
    #print(items)
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [13]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

def cost(theta, labels, lamb):
    unpack(theta)
    #print("cost, len(theta) = ", len(theta))
    predictions = [prediction(d[0], d[1]) for d in rating_data]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [14]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(rating_data)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in rating_data:
        u,i = d[0], d[1]
        pred = prediction(u, i)
        diff = pred - d[2]
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    #print("len [dUserBiases[u] for u in users] = ",len([dUserBiases[u] for u in users])) #13533
    #print("len [dItemBiases[i] for i in items] = ",len([dItemBiases[i] for i in items])) #163899 ???
    #print("len(items) = ",len(items)) 
    #print("derivative, len(dtheta) = ", len(dtheta))
    return np.hstack((dalpha,[dUserBiases[u] for u in users],[dItemBiases[i] for i in items]))

In [15]:
import scipy
import numpy 
theta = np.hstack(([alpha],[0]*(nUsers+nItems)))
#print(len(theta)) 164996
labels = [d[2] for d in rating_data]
lamb = 1
scipy.optimize.fmin_l_bfgs_b(cost, theta, derivative, args = (labels, lamb))

MSE = 0.9008923295603781
MSE = 0.8880111212499086
MSE = 0.90075870005668
MSE = 0.9007586515538737


(array([ 4.58067116e+00, -4.78253046e-05, -6.45546452e-06, ...,
         8.38808750e-07,  8.38537481e-07,  8.38151494e-07]),
 0.9008253818180506,
 {'grad': array([ 5.34144372e-07, -1.97833492e-07, -7.32105272e-09, ...,
         -1.18395761e-10, -1.14568445e-10, -1.09296871e-10]),
  'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 4,
  'nit': 2,
  'warnflag': 0})

Answer for Problem9:

a. MSE on the validation set :


In [34]:
Image(url= "problem10.png")

In [41]:
userBiases_max = max(userBiases, key= lambda x: userBiases[x])
print("maximum value of userBiases:",userBiases_max, userBiases[userBiases_max])
userBiases_min = min(userBiases, key= lambda x: userBiases[x])
print("minimum value of userBiases:",userBiases_min, userBiases[userBiases_min])
itemBiases_max = max(itemBiases, key= lambda x: itemBiases[x])
print("maximum value of itemBiases:",itemBiases_max, itemBiases[itemBiases_max])
itemBiases_min = min(itemBiases, key= lambda x: itemBiases[x])
print("minimum value of itemBiases:",itemBiases_min, itemBiases[itemBiases_min])


maximum value of userBiases: 32445558 3.7003036684773426e-06
minimum value of userBiases: 70705426 -1.2678107941701826e-06
maximum value of itemBiases: 98124873 1.8437616886670768e-07
minimum value of itemBiases: 29147042 -2.62689645997895e-07


Answer for Problem10:

a. user ID that have the largest value of beta : 32445558, value : 3.7003036684773426e-06

b. user ID that have the smallest value of beta : 70705426, value : -1.2678107941701826e-06

c. recipe ID that have the largest value of beta : 98124873, value : 1.8437616886670768e-07

d. recipe ID that have the smallest value of beta : 29147042, value : -2.62689645997895e-07

In [35]:
Image(url= "problem11.png")

In [16]:
# MSE: 0.4547707140445709 -> MSE: 0.45477062463760376

for lamb in [0.00001, 0.0001] :
    print("lamb = ",lamb)
    scipy.optimize.fmin_l_bfgs_b(cost, theta, derivative, args = (labels, lamb))
    
    predictions = open("predictions_Rated_bias_%f.txt" % lamb,'w')
    for l in open("stub_Rated.txt"):
        if l.startswith("user_id"):
            #header
            predictions.write(l)
            continue
        u,i = l.strip().split('-')
        pred = alpha
        if u in userBiases.keys():
            pred+=userBiases[u]
        if i in itemBiases.keys():
            pred+=itemBiases[i]
        predictions.write(u + '-' + i + ",%f\n"%pred)
    predictions.close()


lamb =  1e-05
MSE = 0.9008923295603781
MSE = 0.8880111212499086
MSE = 1.069611774925789
MSE = 0.8854287133812021
MSE = 0.8800495326243006
MSE = 0.8792288815381499
MSE = 0.8761198198963718
MSE = 0.8552503387512107
MSE = 0.844030763897731
MSE = 0.8287385168931087
MSE = 0.8180375474910421
MSE = 0.7979963501373877
MSE = 0.7840140130811376
MSE = 0.7711449292816135
MSE = 0.7613025945278198
MSE = 0.7507671736011382
MSE = 5.345930082893958
MSE = 0.7506959492153554
MSE = 0.7467219604102234
MSE = 0.7387969604882767
MSE = 0.7338251748249827
MSE = 0.7261427552356622
MSE = 0.7185612868667165
MSE = 0.7098738719307657
MSE = 0.7009736748541021
MSE = 0.6902352307829346
MSE = 0.6820939765389208
MSE = 0.6809004970478701
MSE = 0.6806377856494903
MSE = 0.6800865674487122
MSE = 0.678011146489186
MSE = 0.6759285824177541
MSE = 0.6724558791953841
MSE = 0.6714563598671184
MSE = 0.6674767145274385
MSE = 0.6684374809302834
MSE = 0.6645768392321414
MSE = 0.6613946514295945
MSE = 0.6588469783561169
MSE = 0.6552680

MSE = 0.634145990024387
MSE = 0.6341634033962597
MSE = 0.6341469348284917
MSE = 0.6341452700570915
MSE = 0.6341473471083766
MSE = 0.6341167133131254
MSE = 0.634141858898768
MSE = 0.6341444278273037
MSE = 0.6341526355067247
MSE = 0.634154020626509
MSE = 0.634150240106463
MSE = 0.6341477487661004
MSE = 0.634145570355057
MSE = 0.6341447120348698
MSE = 0.6341463156870584
MSE = 0.6341380505559471
MSE = 0.6341449388211792
MSE = 0.6341481788010801
MSE = 0.634154497656838
MSE = 0.6341558386685454
MSE = 0.6341637617328665
MSE = 0.634150696594303
MSE = 0.6341612331188466
MSE = 0.6341645257273949
MSE = 0.6341671913991425
MSE = 0.6341687176764904
MSE = 0.6341680289471773
MSE = 0.6341742650069225
MSE = 0.634177277246967
MSE = 0.6341978263757083
MSE = 0.6341859919882981
MSE = 0.6341923920606669
MSE = 0.6341900242523433
MSE = 0.6341886618744861
MSE = 0.6341885776394552
MSE = 0.6341829802146746
MSE = 0.6341906479203796
MSE = 0.6341951836312085
MSE = 0.6342176687400954
MSE = 0.6341981711512272
MSE = 0.

KeyboardInterrupt: 

In [17]:
for lamb in [0.00001, 0.0001, 0.001] :
    print("lamb = ",lamb)
    scipy.optimize.fmin_l_bfgs_b(cost, theta, derivative, args = (labels, lamb))
    
    predictions = open("predictions_Rated_bias_%f_20000.txt" % lamb,'w')
    for l in open("stub_Made.txt"):
        if l.startswith("user_id"):
            #header
            predictions.write(l)
            continue
        u,i = l.strip().split('-')
        pred = alpha
        if u in userBiases.keys():
            pred+=userBiases[u]
        if i in itemBiases.keys():
            pred+=itemBiases[i]
        predictions.write(u + '-' + i + ",%f\n"%pred)
    predictions.close()


lamb =  1e-05
MSE = 0.9008923295603781
MSE = 0.8880111212499086
MSE = 1.069611774925789
MSE = 0.8854287133812021
MSE = 0.8800495326243006
MSE = 0.8792288815381499
MSE = 0.8761198198963718
MSE = 0.8552503387512107
MSE = 0.844030763897731
MSE = 0.8287385168931087
MSE = 0.8180375474910421
MSE = 0.7979963501373877
MSE = 0.7840140130811376
MSE = 0.7711449292816135
MSE = 0.7613025945278198
MSE = 0.7507671736011382
MSE = 5.345930082893958
MSE = 0.7506959492153554
MSE = 0.7467219604102234
MSE = 0.7387969604882767
MSE = 0.7338251748249827
MSE = 0.7261427552356622
MSE = 0.7185612868667165
MSE = 0.7098738719307657
MSE = 0.7009736748541021
MSE = 0.6902352307829346
MSE = 0.6820939765389208
MSE = 0.6809004970478701
MSE = 0.6806377856494903
MSE = 0.6800865674487122
MSE = 0.678011146489186
MSE = 0.6759285824177541
MSE = 0.6724558791953841
MSE = 0.6714563598671184
MSE = 0.6674767145274385
MSE = 0.6684374809302834
MSE = 0.6645768392321414
MSE = 0.6613946514295945
MSE = 0.6588469783561169
MSE = 0.6552680

MSE = 0.634145990024387
MSE = 0.6341634033962597
MSE = 0.6341469348284917
MSE = 0.6341452700570915
MSE = 0.6341473471083766
MSE = 0.6341167133131254
MSE = 0.634141858898768
MSE = 0.6341444278273037
MSE = 0.6341526355067247
MSE = 0.634154020626509
MSE = 0.634150240106463
MSE = 0.6341477487661004
MSE = 0.634145570355057
MSE = 0.6341447120348698
MSE = 0.6341463156870584
MSE = 0.6341380505559471
MSE = 0.6341449388211792
MSE = 0.6341481788010801
MSE = 0.634154497656838
MSE = 0.6341558386685454
MSE = 0.6341637617328665
MSE = 0.634150696594303
MSE = 0.6341612331188466
MSE = 0.6341645257273949
MSE = 0.6341671913991425
MSE = 0.6341687176764904
MSE = 0.6341680289471773
MSE = 0.6341742650069225
MSE = 0.634177277246967
MSE = 0.6341978263757083
MSE = 0.6341859919882981
MSE = 0.6341923920606669
MSE = 0.6341900242523433
MSE = 0.6341886618744861
MSE = 0.6341885776394552
MSE = 0.6341829802146746
MSE = 0.6341906479203796
MSE = 0.6341951836312085
MSE = 0.6342176687400954
MSE = 0.6341981711512272
MSE = 0.

In [ ]:
for lamb in [100, 1000] :
    print("lamb = ",lamb)
    scipy.optimize.fmin_l_bfgs_b(cost, theta, derivative, args = (labels, lamb))
    
    predictions = open("predictions_Rated_bias_%f.txt" % lamb,'w')
    for l in open("stub_Rated.txt"):
        if l.startswith("user_id"):
            #header
            predictions.write(l)
            continue
        u,i = l.strip().split('-')
        pred = alpha
        if u in userBiases.keys():
            pred+=userBiases[u]
        if i in itemBiases.keys():
            pred+=itemBiases[i]
        predictions.write(u + '-' + i + ",%f\n"%pred)
    predictions.close()

In [58]:
import numpy as np
user_bias_mean = np.array(list(userBiases.values())).mean()
item_bias_mean = np.array(list(itemBiases.values())).mean()
print(user_bias_mean)
print(item_bias_mean)

-8.76262562739984e-25
-7.549711999005067e-26


Answer for Problem11:

a. value lamb wrt MSE :

lamb = 1e-05, MSE = 0.6343308205994943
lamb =  0.0001, MSE = 0.7800174629415336
lamb =  0.001, MSE = 0.8574004686509782
lamb = 0.01, MSE = 0.8911307388311662
lamb =  0.1, MSE = 0.8996097642865012
lamb = 10, MSE = 0.9008789032254846
lamb =  100, MSE = 0.9008909863002668
lamb = 1000, MSE = 0.900892195231244